# 1) 필요한 모듈 import

In [1]:
import cv2
import numpy as np
import math
import time
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2
from keras.models import load_model

In [2]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing_styles = mp.solutions.drawing_styles
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

In [3]:
land_list = [46, 53, 52, 55, 65, 276, 283, 282, 295, 285, 
             130, 160, 158, 133, 153, 144, 
             359, 387, 385, 362, 380, 373,
             168, 6, 195, 4, 61, 39, 0, 269, 291, 405, 17, 181, 234, 132, 152, 288, 454]

# 2) 사용자 EAR & AWM 값 계산

In [4]:
text = "Calculating the average width of your eyes & mouth..."

In [5]:
def calc_ear(p_list):
    a_x = (p_list[0].x - p_list[3].x) ** 2 * 1000
    a_y = (p_list[0].y - p_list[3].y) ** 2 * 1000
    a = a_x + a_y
            
    b_x = (p_list[1].x - p_list[5].x) ** 2 * 1000
    b_y = (p_list[1].y - p_list[5].y) ** 2 * 1000
    b = b_x + b_y
            
    c_x = (p_list[2].x - p_list[4].x) ** 2 * 1000
    c_y = (p_list[2].y - p_list[4].y) ** 2 * 1000
    c = c_x + c_y
    
    return (b + c) / (2 * a)

In [11]:
def calc_awm(p_list):
    a_x = (p_list[0].x - p_list[2].x) ** 2 * 1000
    a_y = (p_list[0].y - p_list[2].y) ** 2 * 1000
    a = a_x + a_y
            
    b_x = (p_list[1].x - p_list[3].x) ** 2 * 1000
    b_y = (p_list[1].y - p_list[3].y) ** 2 * 1000
    b = b_x + b_y
    
    return a * b * math.pi

In [20]:
cap = cv2.VideoCapture(0)
cnt = 0
sum_r = 0
sum_l = 0
sum_m = 0

with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_face_landmarks:
            point_list = []
            for i in land_list:
                point_list.append(results.multi_face_landmarks[0].landmark[i])
            
            sum_r += calc_ear(point_list[10:16])
            sum_l += calc_ear(point_list[16:22])
            sum_m += calc_awm([point_list[26], point_list[28], point_list[30], point_list[32]])
            
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=landmark_pb2.NormalizedLandmarkList(landmark = point_list),
                connection_drawing_spec=drawing_spec)
            
            image = cv2.flip(image, 1)
            cv2.putText(image, text, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (98, 17, 0), 2, cv2.LINE_AA)
            cv2.imshow(text, image)
            cv2.waitKey(1)
            cnt += 1
            
            if cnt == 100:
                break
                
cv2.destroyAllWindows()
cap.release()

In [21]:
mar_r = sum_r / 100
mar_l = sum_l / 100
width_m = sum_m // 100

print("사용자 오른쪽 눈의 평균 EAR:", mar_r)
print("사용자 왼쪽 눈의 평균 EAR:", mar_l)
print("사용자 입술의 평균 크기:", width_m)

사용자 오른쪽 눈의 평균 EAR: 0.1453681094283332
사용자 왼쪽 눈의 평균 EAR: 0.15645403678424058
사용자 입술의 평균 크기: 225.0


# 3) 5분 동안 평균 깜빡임 시간 계산

In [22]:
mode = "Opened"

In [23]:
cap = cv2.VideoCapture(0)
time_start = time.time()
time_close = 0
close_time = []

with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_face_landmarks:
            point_list = []
            for i in land_list:
                point_list.append(results.multi_face_landmarks[0].landmark[i])

            ear_r = calc_ear(point_list[10:16])
            ear_l = calc_ear(point_list[16:22])
                    
            if (ear_r < mar_r * 0.7) and (ear_l < mar_l * 0.7):
                if mode == "Opened ":
                    time_close = time.time()
                mode = "Closed "
            else:
                if mode == "Closed ":
                    close_time.append(time.time() - time_close)
                mode = "Opened "
            
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=landmark_pb2.NormalizedLandmarkList(landmark = point_list),
                connection_drawing_spec=drawing_spec)
            
            image = cv2.flip(image, 1)
            cv2.putText(image, mode, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (98, 17, 0), 2, cv2.LINE_AA)
            cv2.imshow("Recording...", image)
            cv2.waitKey(1)
            
            if time.time() - time_start >= 300:
                break

cv2.destroyAllWindows()
cap.release()

In [24]:
dur_close = sum(close_time) / len(close_time)
print("깜빡임 지속 시간 평균 :", dur_close)

깜빡임 지속 시간 평균 : 0.18467197797994697


# 4) 실제 운전 중인 상태를 가정하여 졸음을 판단

In [25]:
model = load_model('model/model03')

In [27]:
message = ""

# 1)
cName = "Non-Drowsy"
score1 = 0

# 2)
mode_e = "Opened"
warning = ""

flag_e = True
time_close = 0
time_list = []

avg_time = 0
score2 = 0

# 3)
avg_time = dur_close
score3 = 0

# 4)
mode_y = "X"
flag_y = False
score4 = 0

cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    time_close = time.time()
    time_e = time.time()
    time_y = time.time()
    
    while cap.isOpened():
        success, image = cap.read()
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_face_landmarks:
            point_list = []
            for i in land_list:
                point_list.append(results.multi_face_landmarks[0].landmark[i])

            # 1
            temp = []
            for i, points in enumerate(point_list):
                temp.append(points.x)
                temp.append(points.y)
                temp.append(points.z)
            temp = np.array(temp).reshape(1, 117)
        
            score1 = math.floor(model.predict(temp)[0][1] * 5) * 10
            if 30 <= score1:
                cName = "Drowsy"
            else:
                cName = "Non-Drowsy"
                
            # 2
            ear_r = calc_ear(point_list[10:16])
            ear_l = calc_ear(point_list[16:22])
            
            if (ear_r < mar_r * 0.7) and (ear_l < mar_l * 0.7):
                if mode_e == "Opened":
                    time_close = time.time()
                if time.time() - time_close >= 4:
                    warning = "Wake Up!!!"
                elif time.time() - time_close >= 2 and flag_e:
                    score2 += 5 
                    flag_e = False
                mode_e = "Closed"
            else:
                if mode_e == "Closed":
                    warning = ""
                    flag_e = True
                    time_list.append(time.time() - time_close)
                mode_e = "Opened"
                
            # 3
            if time.time() - time_e >= 60:
                avg_time = sum(time_list) / len(time_list)
                time_list = []
                time_e = time.time()
                if avg_time >= dur_close * 2:
                    score3 = 10
                else:
                    score3 = 0
                
            # 4
            width = calc_awm([point_list[26], point_list[28], point_list[30], point_list[32]])
            ratio = width  / width_m
            
            if ratio > 6 and time.time() - time_y >= 2:
                flag_y = True
                mode_y = "O"
            else:
                if flag_y:
                    flag_y = False
                    time_y = time.time()
                    score4 += 10
                    mode_y = "X"
                            
            total = score1 + score2 + score3 + score4
            if total >= 100:
                message = "I think the driver is drowsy"

            image = cv2.flip(image, 1)
            cv2.putText(image, "Face : " + cName, (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, "Eyes : " + mode_e, (30, 75), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, "Avg : " + str(int(avg_time * 1000)) + "(ms)", (30, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, "Yawn : " + mode_y, (30, 145), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1, cv2.LINE_AA)
            
            cv2.putText(image, str(score1), (550, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(image, str(int(score2)), (550, 75), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(image, str(score3), (550, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(image, str(score4), (550, 145), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
            
            cv2.putText(image, warning, (240, 320), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.putText(image, message, (140, 360), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                        
            cv2.imshow('Test', image)
            key = cv2.waitKey(1) & 0xFF
            if key == 27:
                break
            elif key == 32:
                message = ""
                score2 = 0
                score3 = 0
                score4 = 0
                
cv2.destroyAllWindows()
cap.release()

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 17ms/step
